In [99]:
import pandas as pd
import numpy as np
from feature_engine import imputation
from feature_engine.encoding import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder as OHpre

In [100]:
df = pd.read_csv('data/z_train.csv')

In [101]:
df

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,21651,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15321,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
15322,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
15323,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
15324,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [102]:
y = df.target

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15326 non-null  int64  
 1   city                    15326 non-null  object 
 2   city_development_index  15326 non-null  float64
 3   gender                  11725 non-null  object 
 4   relevent_experience     15326 non-null  object 
 5   enrolled_university     15024 non-null  object 
 6   education_level         14957 non-null  object 
 7   major_discipline        13089 non-null  object 
 8   experience              15276 non-null  object 
 9   company_size            10592 non-null  object 
 10  company_type            10435 non-null  object 
 11  last_new_job            14987 non-null  object 
 12  training_hours          15326 non-null  int64  
 13  target                  15326 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [104]:
df.drop('target',axis=1,inplace=True)

In [105]:
df.city.value_counts(dropna=False)

city_103    3497
city_21     2156
city_16     1259
city_114    1067
city_160     670
            ... 
city_129       2
city_8         2
city_140       1
city_171       1
city_121       1
Name: city, Length: 123, dtype: int64

In [106]:
missing_ind = imputation.AddMissingIndicator()

In [107]:
df = missing_ind.fit_transform(df)

In [108]:
df

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,...,last_new_job,training_hours,gender_na,enrolled_university_na,education_level_na,major_discipline_na,experience_na,company_size_na,company_type_na,last_new_job_na
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,...,1,36,0,0,0,0,0,1,1,0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,...,>4,47,0,0,0,0,0,0,0,0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,...,never,83,1,0,0,0,0,1,1,0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,...,never,52,1,1,0,0,0,1,0,0
4,21651,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,...,1,24,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15321,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,...,1,42,0,0,0,0,0,1,1,0
15322,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,...,4,52,0,0,0,0,0,1,1,0
15323,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,...,4,44,0,0,0,0,0,0,0,0
15324,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,...,2,97,0,0,0,1,0,0,0,0


In [109]:
df.gender.value_counts(dropna=False)

Male      10577
NaN        3601
Female      998
Other       150
Name: gender, dtype: int64

In [110]:
gender_hot = OHpre(sparse=False,handle_unknown='ignore')
oh_gender_df = pd.DataFrame(gender_hot.fit_transform(df.gender.values.reshape(-1,1))[:,:-1],columns = gender_hot.categories_[0][:-1])
oh_gender_df

,Female,Male,Other
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
...,...,...,...
15321,0.0,1.0,0.0
15322,0.0,1.0,0.0
15323,0.0,1.0,0.0
15324,0.0,1.0,0.0


In [111]:
df = pd.concat([df,oh_gender_df],axis=1)
df

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,...,enrolled_university_na,education_level_na,major_discipline_na,experience_na,company_size_na,company_type_na,last_new_job_na,Female,Male,Other
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,...,0,0,0,0,1,1,0,0.0,1.0,0.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,...,0,0,0,0,0,0,0,0.0,1.0,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,...,0,0,0,0,1,1,0,0.0,0.0,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,...,1,0,0,0,1,0,0,0.0,0.0,0.0
4,21651,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,...,0,0,0,0,1,1,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15321,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,...,0,0,0,0,1,1,0,0.0,1.0,0.0
15322,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,...,0,0,0,0,1,1,0,0.0,1.0,0.0
15323,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,...,0,0,0,0,0,0,0,0.0,1.0,0.0
15324,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,...,0,0,1,0,0,0,0,0.0,1.0,0.0


In [114]:
df.drop('gender',axis=1,inplace=True)

In [112]:
enrollee_id = df.enrollee_id
df.drop('enrollee_id',axis=1,inplace=True)

In [ ]:
df

,city,city_development_index,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,...,enrolled_university_na,education_level_na,major_discipline_na,experience_na,company_size_na,company_type_na,last_new_job_na,Female,Male,Other
0,city_103,0.920,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,...,0,0,0,0,1,1,0,0.0,1.0,0.0
1,city_40,0.776,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,...,0,0,0,0,0,0,0,0.0,1.0,0.0
2,city_21,0.624,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,...,0,0,0,0,1,1,0,0.0,0.0,0.0
3,city_115,0.789,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,...,1,0,0,0,1,0,0,0.0,0.0,0.0
4,city_176,0.764,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,...,0,0,0,0,1,1,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15321,city_173,0.878,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,...,0,0,0,0,1,1,0,0.0,1.0,0.0
15322,city_103,0.920,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,...,0,0,0,0,1,1,0,0.0,1.0,0.0
15323,city_103,0.920,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,...,0,0,0,0,0,0,0,0.0,1.0,0.0
15324,city_65,0.802,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,...,0,0,1,0,0,0,0,0.0,1.0,0.0


In [117]:
df.city

0        city_103
1         city_40
2         city_21
3        city_115
4        city_176
           ...   
15321    city_173
15322    city_103
15323    city_103
15324     city_65
15325     city_67
Name: city, Length: 15326, dtype: object

In [118]:
df.drop('city',axis=1,inplace=True)

In [119]:
df

,city_development_index,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,...,enrolled_university_na,education_level_na,major_discipline_na,experience_na,company_size_na,company_type_na,last_new_job_na,Female,Male,Other
0,0.920,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,...,0,0,0,0,1,1,0,0.0,1.0,0.0
1,0.776,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,...,0,0,0,0,0,0,0,0.0,1.0,0.0
2,0.624,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,...,0,0,0,0,1,1,0,0.0,0.0,0.0
3,0.789,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,...,1,0,0,0,1,0,0,0.0,0.0,0.0
4,0.764,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,...,0,0,0,0,1,1,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15321,0.878,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,...,0,0,0,0,1,1,0,0.0,1.0,0.0
15322,0.920,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,...,0,0,0,0,1,1,0,0.0,1.0,0.0
15323,0.920,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,...,0,0,0,0,0,0,0,0.0,1.0,0.0
15324,0.802,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,...,0,0,1,0,0,0,0,0.0,1.0,0.0


In [122]:
df.relevent_experience.value_counts()

Has relevent experience    11068
No relevent experience      4258
Name: relevent_experience, dtype: int64

In [130]:
label_exp = preprocessing.LabelEncoder()
df.relevent_experience = pd.DataFrame(label_exp.fit_transform(df.relevent_experience),columns=['relevant_experience'])

In [132]:
df.rename({'relevent_experience':'relevant_experience'},axis=1,inplace=True)

In [127]:
df

,city_development_index,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,...,enrolled_university_na,education_level_na,major_discipline_na,experience_na,company_size_na,company_type_na,last_new_job_na,Female,Male,Other
0,0.920,0,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,...,0,0,0,0,1,1,0,0.0,1.0,0.0
1,0.776,1,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,...,0,0,0,0,0,0,0,0.0,1.0,0.0
2,0.624,1,Full time course,Graduate,STEM,5,NaN,NaN,never,83,...,0,0,0,0,1,1,0,0.0,0.0,0.0
3,0.789,1,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,...,1,0,0,0,1,0,0,0.0,0.0,0.0
4,0.764,0,Part time course,Graduate,STEM,11,NaN,NaN,1,24,...,0,0,0,0,1,1,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15321,0.878,1,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,...,0,0,0,0,1,1,0,0.0,1.0,0.0
15322,0.920,0,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,...,0,0,0,0,1,1,0,0.0,1.0,0.0
15323,0.920,0,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,...,0,0,0,0,0,0,0,0.0,1.0,0.0
15324,0.802,0,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,...,0,0,1,0,0,0,0,0.0,1.0,0.0


In [133]:
df.enrolled_university.value_counts(dropna=False)

no_enrollment       11071
Full time course     2990
Part time course      963
NaN                   302
Name: enrolled_university, dtype: int64

In [134]:
df.education_level.value_counts()

Graduate          9265
Masters           3514
High School       1607
Phd                329
Primary School     242
Name: education_level, dtype: int64

In [113]:
y = df.target

missing_ind = imputation.AddMissingIndicator()
df = missing_ind.fit_transform(df)

gender_hot = OHpre(sparse=False,handle_unknown='ignore')
oh_gender_df = pd.DataFrame(gender_hot.fit_transform(df.gender.values.reshape(-1,1))[:,:-1],columns = gender_hot.categories_[0][:-1])
df = pd.concat([df,oh_gender_df],axis=1)
df.drop('gender',axis=1,inplace=True)

enrollee_id = df.enrollee_id
df.drop('enrollee_id',axis=1,inplace=True)

df.drop('city',axis=1,inplace=True)

label_exp = preprocessing.LabelEncoder()
df.relevent_experience = pd.DataFrame(label_exp.fit_transform(df.relevent_experience),columns=['relevant_experience'])
df.rename({'relevent_experience':'relevant_experience'},axis=1,inplace=True)


AttributeError: 'DataFrame' object has no attribute 'target'